In [1]:
import numpy as np
import matplotlib.pyplot as plt
import PIL
import os
import cv2
import skimage.io
import skimage.color
import skimage.filters
import glob
import copy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd drive/MyDrive/BETR_Summer23

/content/drive/MyDrive/BETR_Summer23


# Image Resize

### Find smallest side of green boxes, and resize
Resize image so that that the distance to an image is "standardized".

In [ ]:
!ls data/'Wound Photos by iPhone'/'Day 0'

'Pig 550-Wound 10-Day 0.jpeg'  'Pig 550-Wound 4-Day 0.jpeg'  'Pig 550-Wound 8-Day 0.jpeg'
'Pig 550-Wound 1-Day 0.jpeg'   'Pig 550-Wound 5-Day 0.jpeg'  'Pig 550-Wound 9-Day 0.jpeg'
'Pig 550-Wound 2-Day 0.jpeg'   'Pig 550-Wound 6-Day 0.jpeg'
'Pig 550-Wound 3-Day 0.jpeg'   'Pig 550-Wound 7-Day 0.jpeg'


In [6]:
imgs_ = glob.glob("data/Wound Photos by iPhone/**/*.jpeg", recursive=True)

In [51]:
def find_min(imgs_):
    """
    input: imgs_ a list of paths to images
    output:
        min_: Minimum number of pixels of green-box
        distances" average number of smallest green-box pixels
        fmin_: name of image with min number of pixels in green-box
    """
    distances = 0
    num_dists = 0
    min_ = 10000000000
    history_path = "no_boxes.txt"
    fmin_ = ""
    if os.path.isfile(history_path):
        os.remove(history_path)
    for filepath in imgs_:

        if filepath.endswith(".jpeg"):

            #image = skimage.io.imread(root + os.path.join(folder, filepath))
            image =  cv2.imread(filepath)
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            h,s,v = cv2.split(hsv)

            ##(3) threshold the S channel using adaptive method(`THRESH_OTSU`)
            th, thresh_img = cv2.threshold(s, 100, 255, cv2.THRESH_OTSU|cv2.THRESH_BINARY)
            #image = image[:,:,1]


            cnts = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            #for cnt in cnts:
            #    approx = cv2.contourArea(cnt)
            #    print(approx)



            #cv2.waitKey()
            min_area = 10000
            max_area = 100000
            image_number = 0
            areas = []

            dist = []
            padding = 10
            regions = []
            image = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
            out = copy.deepcopy(image)

            #plt.imshow( image)
            #plt.show()
            for c in cnts:
                area = cv2.contourArea(c)
                if area > min_area and area < max_area:
                    #print(area)
                    areas.append(area)
                    x,y,w,h = cv2.boundingRect(c)
                    if 0.8<= w/h and w/h <= 1.2:
                        #ROI = out[y:y+h, x:x+w]
                        #cv2.imwrite('ROI_{}.png'.format(image_number), ROI)
                        cv2.rectangle(out, (x, y), (x + w, y + h), (36,255,12), 6)
                        roi = copy.deepcopy(image[y-padding:y+h+padding,
                                        x-padding:x+w+padding])
                        regions.append(roi)

            if len(regions) != 0:
                print(f"number of ROIs: {len(regions)}")
            else:
                print(f"no ROIs found ...")
            greens = []

            for roi in regions:
                pixels = np.float32(roi.reshape(-1, 3))
                if pixels.shape[0] != 0:

                    n_colors = 1
                    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
                    flags = cv2.KMEANS_RANDOM_CENTERS
                    _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
                    _, counts = np.unique(labels, return_counts=True)
                    dominant = palette[np.argmax(counts)]
                    #print(dominant, 95 - dominant[2])
                    #if dominant[0] < 60:
                    if dominant[1] > dominant[0] and dominant[1] > dominant[2] and dominant[2] < 140 and dominant[0] < 140:
                        #dist.append( abs(95 - dominant[1]  ) )
                        greens.append(roi)

            #plt.figure(figsize=(6,6))
            #plt.imshow(regions[idx])
            #plt.show()
            #print(len(greens))
            gcount = 1
            if len(greens) == 0:
            #write results to history file
                with open(history_path, 'a') as file:
                    file.write(f"{filepath}: No box founds \tLine 115\n")
                print("\tNo box found...")
                continue
            else:
                print(f"\tNum boxes: {len(greens)}")
            for x in range(len(greens)):
                #plt.figure(figsize=(2,2))
                #print(type(greens[x]))



                #print(greens[x].mean(axis=0).mean(axis=0))
                plt.imshow(greens[x])
                plt.show()

                mask = np.zeros(greens[x].shape, dtype=np.uint8)

                blur = cv2.GaussianBlur(greens[x], (5,5), 0)

                #plt.imshow(blur)

                # convert to LAB space
                lab = cv2.cvtColor(blur, cv2.COLOR_BGR2LAB)

                # store the a-channel
                a_channel = lab[:,:,1]

                th = cv2.threshold(a_channel,127,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]

                #edge = cv2.Canny(th, 100, 200)
                #(cnts, _) = cv2.findContours(edge.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                # Find distorted rectangle contour and draw onto a mask
                cnts = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cnts = cnts[0] if len(cnts) == 2 else cnts[1]
                rect = cv2.minAreaRect(cnts[0])
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                cv2.drawContours(image,[box],0,(36,255,12),2)
                cv2.fillPoly(mask, [box], (255,255,255))
                # print(box)

                # Find corners on the mask
                mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
                corners = cv2.goodFeaturesToTrack(mask, maxCorners=4, qualityLevel=0.5, minDistance=50)

                gcount +=1
                ccount = 1
                if corners is not None:
                    print(f"\t\tNum corners: {len(corners)}")
                    if len(corners) < 2:
                        with open(history_path, 'a') as file:
                            file.write(f"{filepath}: One corner \tLine 178\n")
                        print("\t\t\tOne corner...")
                        """
                        for corner in corners:
                            p1,p2 = corner.ravel()
                            p1,p2 = int(p1), int(p2)
                            cv2.circle(greens[x],(p1,p2),8,(255,120,255),-1)
                            #print("({}, {})".format(p1,p2))
                            print(f"\t\tcorner {ccount}/{len(corners)}")
                            ccount += 1
                        plt.imshow(th)
                        plt.show()
                        plt.imshow(greens[x])
                        plt.show()
                        """

                    elif len(corners) > 2 and len(corners) < 5:
                        """
                        for corner in corners:
                            p1,p2 = corner.ravel()
                            p1,p2 = int(p1), int(p2)
                            cv2.circle(greens[x],(p1,p2),8,(255,120,255),-1)
                            #print("({}, {})".format(p1,p2))
                            print(f"\t\tcorner {ccount}/{len(corners)}")
                            ccount += 1
                        """
                        tmp = 0
                        for i in range(len(corners[0:-1])):
                            x = np.linalg.norm(corners[i+1]-corners[i])
                            tmp += x
                            if x < min_:
                                min_ = float(x)
                                fmin_ = filepath
                        x = np.linalg.norm(corners[0]-corners[-1])
                        tmp += np.linalg.norm(corners[0]-corners[-1])
                        #print(len(corners))
                        if x < min_:
                            min_ = float(x)
                            fmin_ = filepath
                        num_dists += len(corners)
                        print(f"\tavg. dist: {distances}, min: {min_}")

                        distances = distances * ( num_dists - len(corners) ) / num_dists + tmp / num_dists
                        #dist = numpy.linalg.norm(a-b)
                        """
                        tmp = 0
                        tmp += np.linalg.norm(corners[0]-corners[3])
                        tmp += np.linalg.norm(corners[1]-corners[0])
                        tmp += np.linalg.norm(corners[2]-corners[1])
                        tmp += np.linalg.norm(corners[3]-corners[2])
                        num_dists += 4
                        distances = distances * ( num_dists-4 ) / num_dists + tmp / num_dists
                        print(np.linalg.norm(corners[1]-corners[0]))
                        print(np.linalg.norm(corners[2]-corners[1]))
                        print(np.linalg.norm(corners[3]-corners[2]))
                        print(np.linalg.norm(corners[0]-corners[3]))
                        print(distances)
                        """
                    elif len(corners) > 5:
                        """
                        print(f"\t\t\t{len(corners)} corners...")
                        for corner in corners:
                            p1,p2 = corner.ravel()
                            p1,p2 = int(p1), int(p2)
                            cv2.circle(greens[x],(p1,p2),8,(255,120,255),-1)
                            #print("({}, {})".format(p1,p2))
                            print(f"\t\tcorner {ccount}/{len(corners)}")
                            ccount += 1
                        plt.imshow(th)
                        plt.show()
                        plt.imshow(greens[x])
                        plt.show()
                        """
                        print(f"{len(corners)} found")
                        with open(history_path, 'a') as file:
                            file.write(f"{filepath}: {len(corners)} \tfound Line 237\n")
                else:
                    print("Num Corners:", 0)

                    with open(history_path, 'a') as file:
                        file.write(f"{filepath}: 0 corners \tLine 255\n")

                """
                total = 0
                for c in cnts:

                rect = cv2.minAreaRect(c)
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                cv2.drawContours(greens[x],[box],0,(0,0,255),2)


                epsilon = 0.08 * cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, epsilon, True)

                cv2.drawContours(greens[x], [approx], -1, (0, 255, 0), 4)
                total += 1
                """


                # Mask the result with the original image
                # masked = cv2.bitwise_and(greens[x], greens[x], mask = th)
                # plt.imshow(masked)
                # plt.show()
            #plt.figure(figsize=(16,16))
            #plt.imshow(out)
    return min_, distances, fmin_

In [52]:
min_, avg_, fmin_ = find_min(imgs_)
print(min_, avg_, fmin_)

Output hidden; open in https://colab.research.google.com to view.

In [9]:
#min_ = 121.39604949951172

### Resize images

In [26]:
import csv
bads = "no_boxes.txt"
bad_boxes = []

with open(bads) as csvfile:
    csv_ = csv.reader(csvfile, delimiter=',')
    next(csv_)
    #for row in csv_:
    for row in csv_:
        bad_boxes.append(row[0].split(":")[0])
print(len(bad_boxes))

9


In [27]:
bads

'no_boxes.txt'

In [28]:
# Save Util
def save(img, root, fname):
    os.makedirs(root, exist_ok=True)
    path_near = os.path.join(root, fname)
    out = PIL.Image.fromarray(img)
    out.save(path_near, "jpeg")

In [49]:
imgs_cp = []
def resize(imgs_, save_dir):
    # For Day Zero
    img_count = 0
    for filepath in imgs_:
        if filepath.endswith(".jpeg") and filepath not in bad_boxes:
            print("Reading:", filepath)

            #image = skimage.io.imread(root + os.path.join(folder, filepath))
            image =  cv2.imread(filepath)
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            h,s,v = cv2.split(hsv)

            ##(3) threshold the S channel using adaptive method(`THRESH_OTSU`)
            th, thresh_img = cv2.threshold(s, 100, 255, cv2.THRESH_OTSU|cv2.THRESH_BINARY)
            #image = image[:,:,1]


            cnts = cv2.findContours(thresh_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]

            # wound day 0 WORKS!
            min_area = 10000
            max_area = 100000
            image_number = 0
            areas = []

            dist = []
            padding = 10
            regions = []
            image = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
            out = copy.deepcopy(image)
            clean_out = copy.deepcopy(image)
            #plt.imshow( image)
            #plt.show()
            for c in cnts:
                area = cv2.contourArea(c)
                if area > min_area and area < max_area:
                    #print(area)
                    areas.append(area)
                    x,y,w,h = cv2.boundingRect(c)
                    if 0.8<= w/h and w/h <= 1.2:
                        #ROI = out[y:y+h, x:x+w]
                        #cv2.imwrite('ROI_{}.png'.format(image_number), ROI)
                        cv2.rectangle(out, (x, y), (x + w, y + h), (36,255,12), 6)
                        roi = copy.deepcopy(image[y-padding:y+h+padding,
                                        x-padding:x+w+padding])
                        regions.append(roi)

            if len(regions) != 0:
                print(f"number of ROIs: {len(regions)}")
            else:
                print(f"no ROIs found ...")
            greens = []

            for roi in regions:
                pixels = np.float32(roi.reshape(-1, 3))
                if pixels.shape[0] != 0:

                    n_colors = 1
                    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, .1)
                    flags = cv2.KMEANS_RANDOM_CENTERS
                    _, labels, palette = cv2.kmeans(pixels, n_colors, None, criteria, 10, flags)
                    _, counts = np.unique(labels, return_counts=True)
                    dominant = palette[np.argmax(counts)]
                    #print(dominant, 95 - dominant[2])
                    #if dominant[0] < 60:
                    if dominant[1] > dominant[0] and dominant[1] > dominant[2] and dominant[2] < 140 and dominant[0] < 140:
                        #dist.append( abs(95 - dominant[1]  ) )
                        greens.append(roi)

            if len(greens) == 0:
            #write results to history file
                print("\tNo box found...")
                continue
            else:
                print(f"\tNum boxes: {len(greens)}")
            cur_min = 100000000
            for x in range(len(greens)):

                mask = np.zeros(greens[x].shape, dtype=np.uint8)

                blur = cv2.GaussianBlur(greens[x], (5,5), 0)

                # convert to LAB space
                lab = cv2.cvtColor(blur, cv2.COLOR_BGR2LAB)

                # store the a-channel
                a_channel = lab[:,:,1]

                th = cv2.threshold(a_channel,127,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]

                # Find distorted rectangle contour and draw onto a mask
                cnts = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cnts = cnts[0] if len(cnts) == 2 else cnts[1]
                rect = cv2.minAreaRect(cnts[0])
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                cv2.drawContours(image,[box],0,(36,255,12),2)
                cv2.fillPoly(mask, [box], (255,255,255))
                # print(box)

                # Find corners on the mask
                mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
                corners = cv2.goodFeaturesToTrack(mask, maxCorners=4, qualityLevel=0.5, minDistance=50)

                if corners is not None:
                    print(f"\t\tNum corners: {len(corners)}")
                    if len(corners) < 2:
                        print("\t\t\tOne corner...")


                    elif len(corners) > 1 and len(corners) < 5:

                        tmp = 0
                        for i in range(len(corners[0:-1])):
                            x = np.linalg.norm(corners[i+1]-corners[i])
                            tmp += x
                            if x < cur_min:
                                cur_min = float(x)
                        x = np.linalg.norm(corners[0]-corners[-1])
                        tmp += np.linalg.norm(corners[0]-corners[-1])
                        #print(len(corners))
                        if x < cur_min:
                            cur_min = float(x)

                    elif len(corners) > 5:

                        print(f"{len(corners)} found")
                else:
                    print("Num Corners:", 0)

            width = int(clean_out.shape[1] * min_ / cur_min)
            height = int(clean_out.shape[0] * min_ / cur_min)
            dim = (width, height)
            if dim == (0,0):
                continue
            print(f"\tResizing ratio: {min_}/{cur_min}={min_/cur_min}")

            # resize image
            resized = cv2.resize(clean_out, dim, interpolation = cv2.INTER_AREA)
            print(f'{filepath} resized Dimensions : {resized.shape}')

            save(resized, save_dir, filepath.split("/")[-1])
            img_count += 1
            imgs_cp.append( filepath.split("/")[-1])
    print(f"Files  copied: {img_count}")

In [50]:
resize(imgs_, "Resized")

Reading: data/Wound Photos by iPhone/Day 0/Pig 550-Wound 3-Day 0.jpeg
number of ROIs: 3
	Num boxes: 1
		Num corners: 4
	Resizing ratio: 121.39604949951172/144.678955078125=0.8390719260721728
data/Wound Photos by iPhone/Day 0/Pig 550-Wound 3-Day 0.jpeg resized Dimensions : (3383, 2537, 3)
Reading: data/Wound Photos by iPhone/Day 0/Pig 550-Wound 9-Day 0.jpeg
number of ROIs: 3
	Num boxes: 2
		Num corners: 4
		Num corners: 4
	Resizing ratio: 121.39604949951172/150.0=0.8093069966634114
data/Wound Photos by iPhone/Day 0/Pig 550-Wound 9-Day 0.jpeg resized Dimensions : (3263, 2447, 3)
Reading: data/Wound Photos by iPhone/Day 0/Pig 550-Wound 1-Day 0.jpeg
number of ROIs: 2
	Num boxes: 1
		Num corners: 4
	Resizing ratio: 121.39604949951172/164.02743530273438=0.7400960045217998
data/Wound Photos by iPhone/Day 0/Pig 550-Wound 1-Day 0.jpeg resized Dimensions : (2984, 2238, 3)
Reading: data/Wound Photos by iPhone/Day 0/Pig 550-Wound 6-Day 0.jpeg
number of ROIs: 2
	Num boxes: 2
		Num corners: 4
		Num 

KeyboardInterrupt: ignored

## Crop Images

In [31]:
imgs_cp

['Pig 550-Wound 3-Day 0.jpeg',
 'Pig 550-Wound 6-Day 0.jpeg',
 'Pig 550-Wound 6-Day 0.jpeg',
 'Pig 550-Wound 7-Day 0.jpeg',
 'Pig 550-Wound 7-Day 0.jpeg',
 'Pig 550-Wound 8-Day 0.jpeg',
 'Pig 550-Wound 10-Day 0.jpeg',
 'Pig 550-Wound 5-Day 0.jpeg',
 'Pig 550-Wound 7-Day 3.jpeg',
 'Pig 550-Wound 2-Day 3.jpeg',
 'Pig 550-Wound 2-Day 3.jpeg',
 'Pig 550-Wound 2-Day 3.jpeg',
 'Pig 550-Wound 5-Day 3.jpeg',
 'Pig 550-Wound 5-Day 3.jpeg',
 'Pig 550-Wound 5-Day 3.jpeg',
 'Pig 550-Wound 4-Day 3.jpeg',
 'Pig 550-Wound 4-Day 3.jpeg',
 'Pig 550-Wound 9-Day 3.jpeg',
 'Pig 550-Wound 9-Day 3.jpeg',
 'Pig 550-Wound 1-Day 3.jpeg',
 'Pig 550-Wound 1-Day 3.jpeg',
 'Pig 550-Wound 3-Day 3.jpeg',
 'Pig 550-Wound 3-Day 3.jpeg',
 'Pig 550-Wound 1-Day 7.jpeg',
 'Pig 550-Wound 1-Day 7.jpeg',
 'Pig 550-Wound 1-Day 7.jpeg',
 'Pig 550-Wound 3-Day 7.jpeg',
 'Pig 550-Wound 3-Day 7.jpeg',
 'Pig 550-Wound 2-Day 7.jpeg']

In [ ]:
ls -l Resized | egrep -c '^-'